In [16]:
import pandas as pd
import numpy as np
import datetime as dt
import folium
from folium.plugins import HeatMap, HeatMapWithTime
from branca.element import Figure

# Import modules from current workspace
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

from crawlers.ivar import IvarCrawler
from crawlers.ipma import IpmaCrawler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Azores Seismic Data

### IPMA

In [17]:
ipma_crawler = IpmaCrawler("azores")
ipma_crawler.crawl()
ipma_df = ipma_crawler.pandify_data()

In [18]:
ipma_df.head()

,date,lat,lon,magnitude,magnitude_type,depth,degree,local,obs_region,source,updated_at
772,2022-04-03 20:51:40,38.679,-28.138,2.0,L,13,<NA>,<NA>,Ilha de S. Jorge (W),IPMA,2022-04-03 22:00:00
771,2022-04-03 18:54:25,38.750,-28.315,2.0,L,6,<NA>,<NA>,W Rosais,IPMA,2022-04-03 22:00:00
770,2022-04-03 16:09:29,38.707,-28.191,2.0,L,0,<NA>,<NA>,Ilha de S. Jorge (W),IPMA,2022-04-03 22:00:00
769,2022-04-03 15:21:32,38.698,-28.184,2.0,L,9,<NA>,<NA>,Ilha de S. Jorge (W),IPMA,2022-04-03 22:00:00
768,2022-04-03 14:56:53,38.695,-28.197,2.5,L,10,<NA>,<NA>,Ilha de S. Jorge (W),IPMA,2022-04-03 22:00:00


### IVAR

In [19]:
ivar_crawler = IvarCrawler()
ivar_crawler.crawl()
ivar_df = ivar_crawler.pandify_data()

In [20]:
ivar_df.head()

,event_id,origin_agency,magnitude,date,lat,lon,sentidos,regioes
539,20220403145633,AZO,2.2,2022-04-03 14:56:53.700,38.690,-28.201,[],[S. Jorge]
538,20220403093831,AZO,3.1,2022-04-03 09:38:29.400,38.576,-29.583,[],[W Faial]
537,20220403085520,AZO,2.1,2022-04-03 08:55:42.400,38.721,-29.073,[],[W Faial]
536,20220403065305,AZO,2.0,2022-04-03 06:53:18.700,38.692,-28.195,[],[S. Jorge]
535,20220403062027,AZO,3.0,2022-04-03 06:20:34.800,38.621,-29.420,[],[W Faial]


# Geospatial Analysis

In [37]:
SAO_JORGE_COORDINATES = [38.674,-28.14]
SAO_JORGE_START_DATE = dt.datetime(2022,3,18, tzinfo=dt.timezone.utc)

In [38]:
st_jorge_crisis_df = ivar_df[ivar_df["time"] > SAO_JORGE_START_DATE]

## Heatmaps

In [40]:
fig = Figure(width=600, height=400)
map_hooray = folium.Map(location=SAO_JORGE_COORDINATES, zoom_start=12)

# List comprehension to make out list of lists
heat_data = list(zip(st_jorge_crisis_df["lat"].tolist(), st_jorge_crisis_df["lon"].tolist()))

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
fig.add_child(map_hooray)

In [46]:
st_jorge_crisis_df["time_bin"] = pd.cut(st_jorge_crisis_df["time"], bins=10, labels=False)

C:\Users\hmbf1\AppData\Local\Temp\ipykernel_37388\3207278558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  st_jorge_crisis_df["time_bin"] = pd.cut(st_jorge_crisis_df["time"], bins=10, labels=False)


In [47]:
fig = Figure(width=600, height=400)
map_hooray = folium.Map(location=SAO_JORGE_COORDINATES, zoom_start=12)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = st_jorge_crisis_df[['lat', 'lon', 'time_bin']]
max_time_bin = st_jorge_crisis_df["time_bin"].max()


# List comprehension to make out list of lists
heat_data = [[[row['lat'],row['lon']] for index, row in heat_df[heat_df['time_bin'] == i].iterrows()] for i in range(max_time_bin+1)]

# Plot it on the map
hm = HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_hooray)

# Display the map
fig.add_child(map_hooray)